# A320 Optimization - AirFrance/KLM Project

Group 5 - Aaron, Faruk, Matthieu, Julien, Nicolas

In [70]:
# Necessary packages
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import *
from processing_py import *
import csv
import pandas as pd

In [71]:
nb_places=174
nb_row=29
nb_column=6

In [72]:
def switch_lin_to_real(lin,nb_row=29,nb_column=6):
    column=lin%nb_column
    if column==0:
        column=nb_column
        row=lin//nb_column
    else:
        row=lin//nb_column+1

    real=(column,row) #colonne, rangée
    return real
    
def switch_real_to_lin(real,nb_row=29,nb_column=6):
    column,row=real[0],real[1]

    lin=(row-1)*nb_column+column
    return lin


## Data Importation

In [73]:
# file name : str
info_file = "data.csv"

# InfoGrpe : dict[int : float]
InfoGrpe = {}
Passagers={}
k=1
# Chargement des donnees
with open(info_file) as InfoFile:
    reader = csv.DictReader(InfoFile)
    for row in reader:

        #Groupe_j
        j = int(row["Numero du groupe"])
        InfoGrpe[j] = {}
        InfoGrpe[j]["Composition"]=[int(row['Femmes']),int(row['Hommes']),int(row['Enfants'])]
        InfoGrpe[j]["Nbre total"]=int(row['Tot'])
        InfoGrpe[j]["Classe"]=row['Classe']
        time_str=row["TransitTime"]
        if time_str=="00:00:00":
            time=10**6
        else:
            time_list=[int(x) for x in time_str.split(":")]
            time=time_list[0]*60+time_list[1]
        InfoGrpe[j]["TransitT"]=time
        InfoGrpe[j]["Passagers"]=[]

        #Passagers_groupe_j

        #Femmes
        k_p=k+int(row['Femmes'])
        for x in range(k,k_p):
            Passagers[x]={}
            Passagers[x]["Groupe"]=j
            Passagers[x]["Poids"]=70
            Passagers[x]["Category"] = 'Woman'
            InfoGrpe[j]["Passagers"].append(x)
        k=k_p

        #Hommes
        k_p+=int(row['Hommes'])
        for x in range(k,k_p):
            Passagers[x]={}
            Passagers[x]["Groupe"]=j
            Passagers[x]["Poids"]=85
            Passagers[x]["Category"] = 'Man'
            InfoGrpe[j]["Passagers"].append(x)

        k=k_p

        #Enfants
        k_p+=int(row['Enfants'])
        for x in range(k,k_p):
            Passagers[x]={}
            Passagers[x]["Groupe"]=j
            Passagers[x]["Poids"]=35
            Passagers[x]["Category"] = 'Child'
            InfoGrpe[j]["Passagers"].append(x)
        k=k_p

InfoGrpedf=pd.DataFrame(InfoGrpe)
Passagersdf=pd.DataFrame(Passagers)
nb_passagers=len(Passagers)

if nb_passagers>nb_places:
    print("Embarquement impossible, capacité de l'avion insuffisante'")

#print(InfoGrpedf)
#print(Passagersdf)
## Définition tableau voisins

V_pr={}
V_snd={}
for pl in range(1,nb_places+1):
    V_pr[pl]={i: 0 for i in range(1,nb_places+1)}
    V_snd[pl]={i: 0 for i in range(1,nb_places+1)}
    real=switch_lin_to_real(pl,nb_row,nb_column)
    column,row=real[0],real[1]
    vois_pr=[]
    vois_snd=[]

    if column==1:
        vois_pr.append(pl+1)
    if column in [2,3,4,5]:
        vois_pr.append(pl-1)
        vois_pr.append(pl+1)
    if column==6:
        vois_pr.append(pl-1)

    if row==1:
        vois_snd.append(pl+6)
    if row==29:
        vois_snd.append(pl-6)
    else:
        vois_snd.append(pl+6)
        vois_snd.append(pl-6)

    for v in vois_pr:
        V_pr[pl][v]=1
    for v in vois_snd:
        V_snd[pl][v]=1

## Model Definition

In [74]:
# m : Model
m = Model("PL")

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

## Définition des variables

#format linéaire
E={}

for pa in range(1,nb_passagers+1): # passagers
    for pl in range(1,nb_places+1): # places
        E[(pa,pl)]=m.addVar(vtype=GRB.BINARY)

'''
#format réél
E_bis={}

for pa in range(1,nb_passagers+1): # passagers
    E_bis[pa]={}
    for row in range(1,1+nb_row): #rangée
        name="row{}".format(row)
        E_bis[pa][name]=m.addVar(vtype=GRB.BINARY)
    for column in range(1,1+nb_column): #colonne
        name="column{}".format(column)
        E_bis[pa][name]=m.addVar(vtype=GRB.BINARY)
'''

'\n#format réél\nE_bis={}\n\nfor pa in range(1,nb_passagers+1): # passagers\n    E_bis[pa]={}\n    for row in range(1,1+nb_row): #rangée\n        name="row{}".format(row)\n        E_bis[pa][name]=m.addVar(vtype=GRB.BINARY)\n    for column in range(1,1+nb_column): #colonne\n        name="column{}".format(column)\n        E_bis[pa][name]=m.addVar(vtype=GRB.BINARY)\n'

In [75]:
# Définiton des variables de voisinnage et contraintes associées
# Using max
print('Définiton des variables de voisinnage et contraintes associées')
V_var_pr={}
V_constr_pr={}
V_var_snd={}
V_constr_snd={}
for k in InfoGrpe.keys():
    print(k)
    grpe=InfoGrpe[k]
    for pa1 in grpe["Passagers"]:
        for pa2 in grpe["Passagers"]:
            for pl1 in range(1,nb_places+1): # places
                for pl2 in range(max(1,pa1-6),min(nb_places+1,pa1+6)):
                    if V_pr[pl1][pl2]:
                        V_var_pr[pa1,pa2,pl1,pl2]=m.addVar(vtype=GRB.INTEGER) #=1 ssi pa1 et pa2 voisins primaires en pl1 et pl2
                        V_constr_pr[pa1,pa2,pl1,pl2]=m.addConstr(V_var_pr[pa1,pa2,pl1,pl2]==E[(pa1,pl1)]+E[(pa2,pl2)])
                    if V_snd[pl1][pl2]:
                        V_var_snd[pa1,pa2,pl1,pl2]=m.addVar(vtype=GRB.INTEGER) #=1 ssi pa1 et pa2 voisins secondaires en pl1 et pl2
                        V_constr_snd[pa1,pa2,pl1,pl2]=m.addConstr(V_var_snd[pa1,pa2,pl1,pl2]==E[(pa1,pl1)]+E[(pa2,pl2)])
                        

Définiton des variables de voisinnage et contraintes associées
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111


### Set constraints

In [76]:
## Définition des contraintes de bases

# chaque passager est assis à une place
print("Add contrainte:chaque passager est assis à une place")
C1={}

for pa in range(1,nb_passagers+1): # passagers
    C1[pa]=m.addConstr(quicksum(E[(pa,pl)] for pl in range(1,nb_places+1))==1)

# chaque siège est occupé par au maximum 1 personne
print("Add contrainte:chaque siège est occupé par au maximum 1 personne")

C2={}

for pl in range(1,nb_places+1): # places
    C2[pl]=m.addConstr(quicksum(E[(pa,pl)] for pa in range(1,nb_passagers+1))<=1)


##  Définition contrainte transit time
#print("Add contrainte:transit time")
#place_lim = nb_places//3
#C3={}
#for pa in range(1,nb_passagers+1): # passagers
#   if InfoGrpe[Passagers[pa]["Groupe"]]["TransitT"]<=5400:    # si transit_time inférieur à 1h30
#       C3[pa] = m.addConstr(quicksum([E[(pa,pl)] for pl in range(place_lim+1,nb_places+1)])==0)




## Définition contrainte centrage

print("Add contrainte:barycentre")

#Définition des limites acceptables
column_range=[13,17]
row_range=[3,4]

#Somme des poids de l'ensemble des passages du vol
somme_poids = sum(Passagers[i]["Poids"] for i in range(1,nb_passagers+1))

#Calcul du barycentre
barycentre_column=0
barycentre_row=0
for pa in range(1,nb_passagers+1): # passagers
    for pl in range(1,nb_places+1): # places
            barycentre_column+=switch_lin_to_real(pl,nb_row,nb_column)[0]*E[(pa,pl)]*Passagers[pa]["Poids"]
            barycentre_row+=switch_lin_to_real(pl,nb_row,nb_column)[1]*E[(pa,pl)]*Passagers[pa]["Poids"]
barycentre_column/=somme_poids
barycentre_row/=somme_poids

by=m.addVar(vtype=GRB.CONTINUOUS)
bx=m.addVar(vtype=GRB.CONTINUOUS)
m.addConstr(barycentre_column==by)
m.addConstr(barycentre_row==bx)


C4={}
C4["sup column"]=m.addConstr(barycentre_column<=4)
C4["inf column"]=m.addConstr(barycentre_column>=3)
C4["sup column"]=m.addConstr(barycentre_row<=17)
C4["inf column"]=m.addConstr(barycentre_row>=13)


Add contrainte:chaque passager est assis à une place
Add contrainte:chaque siège est occupé par au maximum 1 personne
Add contrainte:barycentre


### Set objective function

In [77]:
## Définition de la fonction objectif

Sat=0

for k in InfoGrpe.keys():
    grpe=InfoGrpe[k]
    for pa1 in grpe["Passagers"]:
        for pa2 in grpe["Passagers"]:
            for pl1 in range(1,nb_places+1): # places
                for pl2 in range(max(1,pa1-6),min(nb_places+1,pa1+6)):
                    if V_pr[pl1][pl2]:
                        Sat+=V_var_pr[pa1,pa2,pl1,pl2]
                    if grpe["Nbre total"]>3:
                        if V_snd[pl1][pl2]:
                            Sat+=0.5*V_var_snd[pa1,pa2,pl1,pl2]

m.setObjective(Sat,GRB.MAXIMIZE)

In [78]:
m.optimize()

In [79]:
print(m.Status)

2


## Display Results

In [80]:
class A320:
    
    def __init__(self):

        self.app = App(1709,500)    
        A320_seats = self.app.loadImage('C:\\Users\\Faruk\\Desktop\\code\\optim\\AirFranceKLM\\A320.jpg')
        self.app.image(A320_seats,0,0)
        self.app.redraw()
        self.color = np.random.choice(range(256), size=(174,3))

    def draw_passenger(self,row, column, group, category):

        # Fill with group color
        self.app.fill(self.color[group][0],self.color[group][1],self.color[group][2])
        # Stroke with caterory color
        
        self.app.stroke(0,0,0)
        # Find x,y in the drawing
        if column <= 10:
            x = 322+column*30
        elif column == 11:
            x = 333+column*30
        elif column <= 16:
            x = 342+column*30
        else:
            x = 344+column*30
        if row<=3:
            y = 126+row*20
        else:
            y = 156+row*20

        if category == 'Child':
            self.app.ellipse(x,y,10,10)
        else:
            self.app.ellipse(x,y,15,15)

        self.app.redraw()

    def draw_center_of_gravity(self,bx,by):

        self.app.fill(255,0,0)
        self.app.noStroke()
        self.app.ellipse(342+bx*30,141+by*20,5,5)
        self.app.text('CG',350+bx*30,141+by*20)
        self.app.redraw()

    def show_result(self, E, passengers, bx, by):

        nb_passengers = len(passengers)
        nb_places = 174

        for pa in range(1,nb_passengers+1): # passagers
            for pl in range(1,nb_places+1): # places
                if E[(pa,pl)].x:
                    row, column = switch_lin_to_real(pl)
                    self.draw_passenger(row,column,passengers[pa]["Groupe"],passengers[pa]["Category"])
        
        self.draw_center_of_gravity(bx,by)
        
        self.app.redraw()

In [81]:
A320 = A320()
A320.show_result(E,Passagers,bx.x,by.x)

Starting App...
>> [Jython] Created!



In [82]:
#A320.app.exit()